Goal: create functions to mine data for aggregate stats

In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
con=sqlite3.connect('../players.db')

In [4]:
players_df = pd.read_sql('select * from players',con)

In [5]:
df = pd.read_sql('select * from results',con)

In [6]:
am_players = list(players_df.query('Team == "America"')['Name'])
euro_players = list(players_df.query('Team == "Europe"')['Name'])

In [7]:
d={}
for player in am_players:
    d[player] = df['American_player'].str.contains(player)
for player in euro_players:
    d[player] = df['European_player'].str.contains(player)

In [8]:
#Create shorthands for boolean indexing the dataframe
singles = df['Format']=='Singles'
doubles = df['Format']=='Doubles'
teams = df['Format']=='Teams'
triples = df['Format']=='Triples'
europe_won = df['Europe_won'] == True
america_won = df['Europe_won'] == False

In [9]:
def winlosslist(player, country, df=df):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
        smp=swin+sloss
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]    
        smp=swin+sloss
    swl='{}-{}'.format(swin,sloss)
    try:
        spct=swin*100/(swin+sloss)
    except:
        spct=0
    
    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        dmp=dwin+dloss
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
        dmp=dwin+dloss
    dwl='{}-{}'.format(dwin,dloss)
    try:
        dpct=dwin*100/(dwin+dloss)   
    except:
        dpct=0
    
        #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
        tmp=twin+tloss
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
        tmp=twin+tloss
    twl='{}-{}'.format(twin,tloss)
    try:
        tpct=twin*100/(twin+tloss)   
    except:
        tpct=0
        
        #triples
    if 'a' in country:
        trwin=df[d[player]&america_won&triples].shape[0]
        trloss=df[d[player]&~america_won&triples].shape[0]
        trmp=trwin+trloss
    else:
        trloss=df[d[player]&america_won&triples].shape[0]
        trwin=df[d[player]&~america_won&triples].shape[0] 
        trmp=trwin+trloss
    trwl='{}-{}'.format(trwin,trloss)
    try:
        trpct=round(trwin*100/(trwin+trloss),2)        
    except:
        trpct=0

    #combined
    cwin=dwin+swin+twin+trwin
    closs=dloss+sloss+tloss+trloss
    cmp=cwin+closs
    cwl='{}-{}'.format(cwin,closs)
    cpct=cwin*100/(cwin+closs)
    l=[player,country,smp,swin,sloss,swl,round(spct,2),dmp,dwin,dloss,dwl,round(dpct,2), twl,trwl,cmp, cwin,closs,cwl,round(cpct,2)]
    return(l)

In [10]:
#prepare list of winlosslists for each player to create player stats dataframe
L=[]
for player in am_players:
    try:
        lst = winlosslist(player,'America')
        L.append(lst)
    except: #Zerodivision error:
        print('failure- am: ', player)
#         pass
for player in euro_players:
    try:
        lst = winlosslist(player,'Europe')
        L.append(lst)
    except: #Zerodivision error:
        print('failure -eu ', player)
#         pass

In [11]:
#create player_stats dataframe
colz = 'Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct'.split(',')
player_stats = pd.DataFrame(L, columns = colz)

In [14]:
player_stats[player_stats.Player=='Joshua Filler']

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
76,Joshua Filler,Europe,2,2,0,2-0,100.0,2,2,0,2-0,100.0,1-0,0-0,5,5,0,5-0,100.0


In [15]:
player_stats

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
0,Johnny Archer,America,30,17,13,17-13,56.67,50,27,23,27-23,54.00,5-4,1-1,91,50,41,50-41,54.95
1,Earl Strickland,America,26,14,12,14-12,53.85,36,24,12,24-12,66.67,3-2,2-0,69,43,26,43-26,62.32
2,Shane Van Boening,America,27,11,16,11-16,40.74,28,11,17,11-17,39.29,4-7,0-0,66,26,40,26-40,39.39
3,Rodney Morris,America,18,10,8,10-8,55.56,29,12,17,12-17,41.38,3-5,2-0,57,27,30,27-30,47.37
4,Corey Deuel,America,13,7,6,7-6,53.85,25,13,12,13-12,52.00,5-2,2-0,47,27,20,27-20,57.45
5,Jeremy Jones,America,14,7,7,7-7,50.00,19,10,9,10-9,52.63,0-1,0-0,34,17,17,17-17,50.00
6,Charlie Williams,America,8,6,2,6-2,75.00,13,6,7,6-7,46.15,0-0,0-0,21,12,9,12-9,57.14
7,Dennis Hatch,America,9,4,5,4-5,44.44,14,7,7,7-7,50.00,2-3,0-0,28,13,15,13-15,46.43
8,Nick Varner,America,7,3,4,3-4,42.86,12,5,7,5-7,41.67,0-0,0-0,19,8,11,8-11,42.11
9,Mike Dechaine,America,7,1,6,1-6,14.29,10,2,8,2-8,20.00,2-2,0-0,21,5,16,5-16,23.81


In [17]:
# player_stats.to_sql('playerstats',con,if_exists='replace')